In [1]:
import pandas as pd
import contractions
import demoji
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tarang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
training_df = pd.read_csv('Training Data.csv')
training_df

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-4311,"Leftie Sjws ,simps and women eating it up. So ...",sexist,4. prejudiced discussions,4.2 supporting systemic discrimination against...
1,sexism2022_english-8677,"Thank YOU Lord for favoring me! ""[Whoso] finde...",not sexist,none,none
2,sexism2022_english-19317,Another Democrat with signal eyeglasses.There ...,sexist,2. derogation,2.1 descriptive attacks
3,sexism2022_english-4631,I was 23. I met a girl on tinder and lost it. ...,not sexist,none,none
4,sexism2022_english-650,"Try 13. I witnessed a groups teenage girls, no...",not sexist,none,none
...,...,...,...,...,...
11195,sexism2022_english-18867,How the fuck would they know I'm toxic off not...,not sexist,none,none
11196,sexism2022_english-4039,Illegal immigrants are stronger together. Depo...,not sexist,none,none
11197,sexism2022_english-4011,if your first reaction to being bullied by zoe...,sexist,3. animosity,"3.1 casual use of gendered slurs, profanities,..."
11198,sexism2022_english-7213,END TIME BIRTH PAINS- getting closer together ...,not sexist,none,none


In [3]:
training_df.groupby(by='label_sexist').size()

label_sexist
not sexist    8470
sexist        2730
dtype: int64

In [4]:
training_df.groupby(by='label_category').size()

label_category
1. threats, plans to harm and incitement     241
2. derogation                               1297
3. animosity                                 930
4. prejudiced discussions                    262
none                                        8470
dtype: int64

In [5]:
training_df.groupby(by='label_vector').size()

label_vector
1.1 threats of harm                                                  45
1.2 incitement and encouragement of harm                            196
2.1 descriptive attacks                                             577
2.2 aggressive and emotive attacks                                  566
2.3 dehumanising attacks & overt sexual objectification             154
3.1 casual use of gendered slurs, profanities, and insults          502
3.2 immutable gender differences and gender stereotypes             342
3.3 backhanded gendered compliments                                  49
3.4 condescending explanations or unwelcome advice                   37
4.1 supporting mistreatment of individual women                      55
4.2 supporting systemic discrimination against women as a group     207
none                                                               8470
dtype: int64

In [6]:
training_df['text'] = training_df['text'].str.replace('`', "'") #consistent quotes
training_df['text'] = training_df['text'].str.replace('“', '"') #consistent quotes
training_df['text'] = training_df['text'].str.replace('”', '"') #consistent quotes

for i in range(len(training_df.axes[0])):
    training_df.iat[i,1] = contractions.fix(training_df.iat[i,1]) #remove contractions

In [7]:
training_df['text'] = training_df['text'].str.replace('[0-9]*', "", regex=True) #remove digits
training_df['text'] = training_df['text'].str.replace('\\[USER\\]|\\[URL\\]', "", regex=True) #remove "URL" and "USER"
training_df['text'] = training_df['text'].str.lower() #convert to lowercase

In [8]:
training_df['text'] = training_df['text'].str.replace(r'\u200d', "", regex=True)
training_df['text'] = training_df['text'].str.replace(r'\u200f', "", regex=True)
training_df['text'] = training_df['text'].str.replace(r'\u200b', "", regex=True)

In [9]:
punctuation = string.punctuation + "—"
stop_words = stopwords.words('english')
porter = PorterStemmer()

for i in range(len(training_df.axes[0])):
    text = training_df.iat[i,1]
    text = text.translate(str.maketrans(punctuation, ' '*len(punctuation), '')) #remove punctuations
    text = demoji.replace(text, "")
    text = word_tokenize(text) #tokenize
    text = [word for word in text if word not in stop_words] #remove stopwords
    text = [porter.stem(word) for word in text] #stemming
    training_df.iat[i,1] = text
    
training_df

,rewire_id,text,label_sexist,label_category,label_vector
0,sexism2022_english-4311,"[lefti, sjw, simp, women, eat, basic, major, t...",sexist,4. prejudiced discussions,4.2 supporting systemic discrimination against...
1,sexism2022_english-8677,"[thank, lord, favor, whoso, findeth, wife, fin...",not sexist,none,none
2,sexism2022_english-19317,"[anoth, democrat, signal, eyeglass, way, quali...",sexist,2. derogation,2.1 descriptive attacks
3,sexism2022_english-4631,"[met, girl, tinder, lost, felt, much, better, ...",not sexist,none,none
4,sexism2022_english-650,"[tri, wit, group, teenag, girl, none, could, p...",not sexist,none,none
...,...,...,...,...,...
11195,sexism2022_english-18867,"[fuck, would, know, toxic, noth, pictur, first...",not sexist,none,none
11196,sexism2022_english-4039,"[illeg, immigr, stronger, togeth, deport, mom,...",not sexist,none,none
11197,sexism2022_english-4011,"[first, reaction, bulli, zoey, tuhr, shake, tr...",sexist,3. animosity,"3.1 casual use of gendered slurs, profanities,..."
11198,sexism2022_english-7213,"[end, time, birth, pain, get, closer, togeth, ...",not sexist,none,none


In [10]:
training_df.to_csv('Preprocessed Training Data.csv', index=False)

In [11]:
taskA_df = training_df[['text', 'label_sexist']]
taskB_df = training_df[['text', 'label_category']]
taskC_df = training_df[['text', 'label_vector']]

taskA_df.to_csv('Training Data (TaskA).csv', index=False)
taskB_df.to_csv('Training Data (TaskB).csv', index=False)
taskC_df.to_csv('Training Data (TaskC).csv', index=False)